In [1]:
! pip install pandas
! pip install requests
! pip install plyer 


In [2]:
import pandas as pd
import requests as rq
import plyer 
import datetime
import sqlite3

In [3]:
def error(error_message):
    current_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    message = f'{error_message} {current_date}'
    plyer.notification.notify(
        title='Error',
        message=message,
        app_name="Alerta"
    )

In [4]:
def get_response(url):
    response = rq.get(url)
    if response.status_code == 200:
        data_json = response.json()
        return data_json
    else:
        error('Tabela nao encontrada')

In [5]:
def get_tables(url):
    response = get_response(url)
    return response

In [6]:
def filter_df(df, min_population):
    return df[df['population'] > min_population]

In [7]:
def unstack_df(df, index_col):
    return df.set_index(index_col).unstack()

In [86]:
def create_df(data, columns):
    if data is None or not columns:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df['country'] = df['name'].apply(lambda x: x.get('common', ''))
    if 'capital' in df.columns:
        df['capital'] = df['capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
    print(df)
    df = df[columns]
    df_cleaned = df.dropna()
    return df_cleaned



def filter_df(df):
    def replace_empty_list(val):
        return '' if isinstance(val, list) and len(val) == 0 else val
    
    df_filtered = df.applymap(replace_empty_list)
    return df_filtered


def create_table(table_name, columns, db_name='dados_paises.db'):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    col_defs = ', '.join([f'"{col}" TEXT' for col in columns])
    cursor.execute(f'CREATE TABLE IF NOT EXISTS "{table_name}" ({col_defs})')
    conn.commit()
    conn.close()
    print(f"Table '{table_name}' created successfully.")


def insert_data(table_name, data, db_name='dados_paises.db'):
    conn = sqlite3.connect(db_name,timeout=10)
    cursor = conn.cursor()
    if not data:
        print("No data to insert.")
        return
    
    placeholders = ', '.join(['?'] * len(data[0]))
    for row in data:
        cursor.execute(f'INSERT INTO "{table_name}" VALUES ({placeholders})', row)
    conn.commit()
    conn.close()
    print(f"Data inserted into '{table_name}' successfully.")


def process_data(url, table_name, columns, additional_processing=None):
    data = get_response(url)
    if data:
        df = create_df(data, columns)
        df = filter_df(df)
        create_table(table_name,columns)
        if additional_processing:
            additional_processing(df)
        insert_data(table_name, df.values.tolist())
    else:
        print(f"Failed to fetch data from {url}.")


def process_languages_countries(df, table_name='LanguagesCountries', db_name='dados_paises.db'):
    languages_countries = {}
    for index, row in df.iterrows():
        country_name = row['country']
        languages_info = row['languages']
        if isinstance(languages_info, dict):
            for language_code, language_info in languages_info.items():
                language_name = language_info if isinstance(language_info, str) else language_info.get('name', '')
                if language_name:
                    languages_countries.setdefault(language_name, []).append(country_name)
                    print(languages_countries)
    data_to_insert = [(language, country) for language, countries in languages_countries.items() for country in countries]
    insert_data(table_name, data_to_insert, db_name)

In [88]:
def get_countries():
    url = 'https://restcountries.com/v3.1/all?fields=name,capital,region,subregion,area,population'
    process_data(url, 'Paises', ['country','capital','region','subregion','area','population'])


def get_independent_countries():
    url = 'https://restcountries.com/v3.1/independent?status=true&fields=name,capital,region,subregion,area,population'
    process_data(url, 'Independentes', ['country','capital','region','subregion','area','population'])


def get_languages_countries():
    url = 'https://restcountries.com/v3.1/all?fields=name,languages'
    process_data(url, 'LanguagesCountries', ['languages', 'country'], additional_processing=process_languages_countries)


#get_countries()
#get_independent_countries()
get_languages_countries()


                                                  name  \
0    {'common': 'Moldova', 'official': 'Republic of...   
1    {'common': 'United States', 'official': 'Unite...   
2    {'common': 'Mayotte', 'official': 'Department ...   
3    {'common': 'Nauru', 'official': 'Republic of N...   
4    {'common': 'Mozambique', 'official': 'Republic...   
..                                                 ...   
245  {'common': 'Benin', 'official': 'Republic of B...   
246  {'common': 'Japan', 'official': 'Japan', 'nati...   
247  {'common': 'Dominican Republic', 'official': '...   
248  {'common': 'Qatar', 'official': 'State of Qata...   
249  {'common': 'Gabon', 'official': 'Gabonese Repu...   

                              languages             country  
0                   {'ron': 'Romanian'}             Moldova  
1                    {'eng': 'English'}       United States  
2                     {'fra': 'French'}             Mayotte  
3    {'eng': 'English', 'nau': 'Nauru'}               N

/var/folders/qx/_jjs26m91dx5hnhwzqmp8xbm0000gp/T/ipykernel_46616/4293510471.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df.applymap(replace_empty_list)


ProgrammingError: Error binding parameter 1: type 'dict' is not supported